In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
pd.pandas.set_option('display.max_columns', None)
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import mean_squared_error

In [1]:
X_train = pd.read_csv('X_train_lasso.csv')
display(X_train.head())
print(X_train.shape)

X_test = pd.read_csv('X_test_lasso.csv')
display(X_test.head())
print(X_test.shape)

y_df = pd.read_csv('y_train_transformed.csv', names = ['y_train_transformed'])
display(y_df.head())
print(y_df.shape)

<IPython.core.display.Javascript object>

,Item_Weight,Item_Visibility,Item_MRP,Price_Per_Unit,Outlet_Vintage,Item_Identifier_FD,Item_Fat_Content_Regular,Item_Type_Perishables,Outlet_Size_Medium,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type3,Outlet_Crowd_Low_Crowd
0,-0.409175,-0.921425,1.163142,1.200161,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.828270,-0.812330,-1.031872,-0.324319,-0.588235,-1.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0
2,0.607564,-0.896179,-0.015191,-0.332145,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.818351,-0.703279,0.425652,-0.162868,0.058824,0.0,1.0,1.0,0.0,1.0,-1.0,0.0,1.0
4,-0.455053,-1.031434,-0.970966,-0.622997,0.705882,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0


(8523, 13)


<IPython.core.display.Javascript object>

,Item_Weight,Item_Visibility,Item_MRP,Price_Per_Unit,Outlet_Vintage,Item_Identifier_FD,Item_Fat_Content_Regular,Item_Type_Perishables,Outlet_Size_Medium,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type3,Outlet_Crowd_Low_Crowd
0,1.010539,-1.305060,-0.382832,-0.759623,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.533168,-0.336835,-0.606564,-0.047305,-0.470588,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.247985,0.527794,1.075409,0.507836,0.058824,-1.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0
3,-0.655301,-0.945438,0.130925,0.847099,-0.470588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.123993,0.718598,0.993465,0.560052,0.823529,0.0,1.0,1.0,0.0,1.0,-1.0,1.0,0.0


(5681, 13)


<IPython.core.display.Javascript object>

,y_train_transformed
0,15.515611
1,7.625576
2,12.800240
3,9.013888
4,9.982319


(8523, 1)


In [2]:
y_train = y_df['y_train_transformed']
display(y_train.head())
print(y_train.shape)

0    15.515611
1     7.625576
2    12.800240
3     9.013888
4     9.982319
Name: y_train_transformed, dtype: float64

(8523,)


In [5]:
RMSEs = {}

In [22]:
test = pd.read_csv('BigMart_Test.csv')
display(test.head())

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


#### Ridge

In [121]:
np.logspace(-1, 1, 1000)

array([ 0.1       ,  0.10046204,  0.10092622,  0.10139254,  0.10186102,
        0.10233166,  0.10280447,  0.10327947,  0.10375667,  0.10423607,
        0.10471768,  0.10520152,  0.1056876 ,  0.10617592,  0.1066665 ,
        0.10715934,  0.10765446,  0.10815187,  0.10865158,  0.10915359,
        0.10965793,  0.11016459,  0.1106736 ,  0.11118496,  0.11169868,
        0.11221478,  0.11273326,  0.11325413,  0.11377741,  0.11430311,
        0.11483124,  0.11536181,  0.11589483,  0.11643031,  0.11696827,
        0.11750871,  0.11805165,  0.1185971 ,  0.11914507,  0.11969557,
        0.12024861,  0.12080421,  0.12136238,  0.12192313,  0.12248646,
        0.1230524 ,  0.12362095,  0.12419214,  0.12476596,  0.12534243,
        0.12592156,  0.12650337,  0.12708787,  0.12767507,  0.12826498,
        0.12885762,  0.129453  ,  0.13005113,  0.13065202,  0.13125568,
        0.13186214,  0.1324714 ,  0.13308347,  0.13369837,  0.13431612,
        0.13493671,  0.13556018,  0.13618652,  0.13681576,  0.13

In [7]:
alphas = np.logspace(0, 1, 1000)

ridgecv = RidgeCV(alphas = alphas, cv = 10)
ridgecv.fit(X_train, y_train)

print('Best Alpha Value from RidgeCV:', ridgecv.alpha_)

Best Alpha Value from RidgeCV: 2.2664980792736933


In [8]:
ridge = Ridge(alpha = ridgecv.alpha_)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_train)
RMSEs['Ridge'] = np.sqrt(mean_squared_error(y_train, y_pred))
print(RMSEs['Ridge'])

2.0065378031387846


In [23]:
# Preparing the data for submission
y_test_pred = ridge.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution1_BigMart_Sales_Ridge_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1652.854164
1,FDW14,OUT017,1306.638282
2,NCN55,OUT010,843.272058
3,FDQ58,OUT017,2325.856290
4,FDY38,OUT027,5437.774504


#### Decision Tree

In [9]:
dt = DecisionTreeRegressor(random_state = 0)
dt.fit(X_train, y_train)
y_train_pred = dt.predict(X_train)
RMSEs['DT'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['DT'])

0.0


#### Tuning Decision Tree

In [10]:
dt = DecisionTreeRegressor(random_state = 0)

params = {'max_depth' : sp_randint(2, 100),
         'min_samples_split' : sp_randint(2, 25),
          'min_samples_leaf' : sp_randint(1, 25)}

rsearch = RandomizedSearchCV(dt, param_distributions = params,
                            n_iter = 100, cv = 10, random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 12}


##### Tuned Decision Tree

In [11]:
dt = DecisionTreeRegressor(**rsearch.best_params_, random_state = 0)
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)

RMSEs['DT'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['DT'])

1.9628277120187039


In [24]:
# Preparing the data for submission
y_test_pred = dt.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution2_BigMart_Sales_DT_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1613.794949
1,FDW14,OUT017,1273.751060
2,NCN55,OUT010,563.992404
3,FDQ58,OUT017,2439.609939
4,FDY38,OUT027,5952.320066


### Random Forest

In [12]:
rf = RandomForestRegressor(random_state = 0)

rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)

RMSEs['RF'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['RF'])

0.7806225546239443


#### Tuning Random Forest

In [16]:
rf = RandomForestRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
         'max_features' : sp_randint(1, 13),
         'max_depth' : sp_randint(2, 50),
         'min_samples_leaf' : sp_randint(1, 25),
         'min_samples_split' : sp_randint(2, 25),
         'oob_score' : [False, True]}

rsearch = RandomizedSearchCV(rf, param_distributions = params, n_jobs = -1,
                             n_iter = 100, cv = 5, random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'max_depth': 8, 'max_features': 11, 'min_samples_leaf': 22, 'min_samples_split': 18, 'n_estimators': 144, 'oob_score': True}


##### Tuned Random Forest

In [17]:
rf = RandomForestRegressor(**rsearch.best_params_, random_state = 0)

rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)

RMSEs['RF'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['RF'])

1.888891311648034


In [25]:
# Preparing the data for submission
y_test_pred = rf.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution3_BigMart_Sales_RF_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1478.811575
1,FDW14,OUT017,1262.961004
2,NCN55,OUT010,563.812637
3,FDQ58,OUT017,2255.092557
4,FDY38,OUT027,6016.411302


In [18]:
imp = pd.DataFrame(rf.feature_importances_, index = X_train.columns, columns = ['Importance'])
display(imp.sort_values(by = 'Importance', ascending = False))

,Importance
Outlet_Crowd_Low_Crowd,0.460453
Item_MRP,0.427258
Price_Per_Unit,0.033899
Outlet_Vintage,0.031455
Outlet_Type_Supermarket Type3,0.030885
Item_Visibility,0.006902
Item_Weight,0.004562
Outlet_Type_Supermarket Type1,0.001943
Outlet_Location_Type_Tier 3,0.001101
Item_Fat_Content_Regular,0.000502


### KNN

In [19]:
knn = KNeighborsRegressor()

knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)

RMSEs['KNN'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['KNN'])

1.772456447296481


#### Tuning KNN

In [21]:
knn = KNeighborsRegressor()
params = {'n_neighbors' : sp_randint(1, 25),
         'p' : sp_randint(1, 10)}

rsearch = RandomizedSearchCV(knn, param_distributions = params, 
                             cv = 5, n_iter = 25, n_jobs = -1, 
                             random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'n_neighbors': 20, 'p': 1}


##### Tuned KNN

In [26]:
knn = KNeighborsRegressor(**rsearch.best_params_)

knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)

RMSEs['KNN'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['KNN'])

1.9277843235451884


In [29]:
# Preparing the data for submission
y_test_pred = knn.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution4_BigMart_Sales_KNN_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1846.788620
1,FDW14,OUT017,1281.561710
2,NCN55,OUT010,559.569020
3,FDQ58,OUT017,2089.070203
4,FDY38,OUT027,5115.412768


### AdaBoost

In [27]:
ada = AdaBoostRegressor(random_state = 0)
ada.fit(X_train, y_train)

y_train_pred = ada.predict(X_train)

RMSEs['Ada'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['Ada'])

2.068100931302001


#### Tuning AdaBoost

In [28]:
ada = AdaBoostRegressor(random_state = 0)
params = {'n_estimators' : sp_randint(25, 100),
         'learning_rate' : np.logspace(-1, 0.5, 100),
         'loss' : ['linear', 'square']}

rsearch = RandomizedSearchCV(ada, param_distributions = params, 
                             cv = 5, n_iter = 100, n_jobs = -1,
                             random_state = 0)
rsearch.fit(X_train, y_train)
print(rsearch.best_params_)

{'learning_rate': 0.1873817422860384, 'loss': 'linear', 'n_estimators': 79}


##### Tuned AdaBoost

In [30]:
ada = AdaBoostRegressor(**rsearch.best_params_, random_state = 0)

ada.fit(X_train, y_train)

y_train_pred = ada.predict(X_train)

RMSEs['Ada'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['Ada'])

2.0483415428498435


In [31]:
# Preparing the data for submission
y_test_pred = ada.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution5_BigMart_Sales_ADA_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1460.335173
1,FDW14,OUT017,1287.057296
2,NCN55,OUT010,540.971063
3,FDQ58,OUT017,2343.867826
4,FDY38,OUT027,4687.579244


### Gradient Boosting

In [33]:
gbm = GradientBoostingRegressor(random_state = 0)

gbm.fit(X_train, y_train)
y_train_pred = gbm.predict(X_train)

RMSEs['GBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['GBM'])

1.8977889375561592


#### Tuning GBM

In [45]:
gbm = GradientBoostingRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
          'max_features' : sp_randint(1, 13),
          'max_depth' : sp_randint(2, 50),
          'min_samples_leaf' : sp_randint(1, 25),
          'min_samples_split' : sp_randint(2, 25),
          'learning_rate' : sp_uniform(0, 1)}

rsearch_gbm = RandomizedSearchCV(gbm, param_distributions = params, n_jobs = -1,
                                 cv = 3, n_iter = 25, random_state = 0)

rsearch_gbm.fit(X_train, y_train)
print(rsearch_gbm.best_params_)

{'learning_rate': 0.10204481074802807, 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 21, 'min_samples_split': 10, 'n_estimators': 128}


##### Tuned GBM

In [47]:
gbm = GradientBoostingRegressor(**rsearch_gbm.best_params_, random_state = 0)

gbm.fit(X_train, y_train)
y_train_pred = gbm.predict(X_train)

RMSEs['GBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['GBM'])

1.8086620744899984


In [48]:
# Preparing the data for submission
y_test_pred = gbm.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution6_BigMart_Sales_GBM_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1661.022833
1,FDW14,OUT017,1309.649602
2,NCN55,OUT010,553.706685
3,FDQ58,OUT017,2527.065159
4,FDY38,OUT027,6006.994017


### LightGBM

In [38]:
lgbr = lgb.LGBMRegressor(random_state = 0)

lgbr.fit(X_train, y_train)

y_train_pred = lgbr.predict(X_train)

RMSEs['LGBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['LGBM'])

1.730752030344429


#### Tuning LightGBM

In [39]:
np.logspace(-1, 0.5, 150)

array([0.1       , 0.10234511, 0.10474522, 0.10720162, 0.10971562,
       0.11228858, 0.11492187, 0.11761692, 0.12037517, 0.1231981 ,
       0.12608724, 0.12904413, 0.13207036, 0.13516756, 0.1383374 ,
       0.14158157, 0.14490182, 0.14829993, 0.15177773, 0.15533709,
       0.15897992, 0.16270818, 0.16652388, 0.17042905, 0.17442581,
       0.17851629, 0.1827027 , 0.18698729, 0.19137235, 0.19586025,
       0.2004534 , 0.20515426, 0.20996536, 0.21488929, 0.21992869,
       0.22508626, 0.23036479, 0.23576711, 0.24129612, 0.24695479,
       0.25274616, 0.25867334, 0.26473953, 0.27094797, 0.27730201,
       0.28380506, 0.29046061, 0.29727224, 0.30424362, 0.31137848,
       0.31868066, 0.32615408, 0.33380277, 0.34163082, 0.34964246,
       0.35784197, 0.36623377, 0.37482237, 0.38361238, 0.39260853,
       0.40181565, 0.41123868, 0.4208827 , 0.43075288, 0.44085452,
       0.45119307, 0.46177406, 0.47260319, 0.48368627, 0.49502926,
       0.50663826, 0.51851951, 0.53067938, 0.54312442, 0.55586

In [40]:
lgbr = lgb.LGBMRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
         'max_depth' : sp_randint(1, 25),
         'learning_rate' : sp_uniform(0, 1)}

rsearch_lgbm = RandomizedSearchCV(lgbr, param_distributions = params, n_jobs = -1,
                                 cv = 5, n_iter = 100, random_state = 0)

rsearch_lgbm.fit(X_train, y_train)
print(rsearch_lgbm.best_params_)

{'learning_rate': 0.0923738894608982, 'max_depth': 2, 'n_estimators': 168}


##### Tuned LightGBM

In [41]:
lgbr = lgb.LGBMRegressor(**rsearch_lgbm.best_params_, random_state = 0)

lgbr.fit(X_train, y_train)

y_train_pred = lgbr.predict(X_train)

RMSEs['LGBM'] = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(RMSEs['LGBM'])

1.9244315314917697


In [42]:
# Preparing the data for submission
y_test_pred = lgbr.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution7_BigMart_Sales_LGBR_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1372.953640
1,FDW14,OUT017,1336.779541
2,NCN55,OUT010,560.657179
3,FDQ58,OUT017,2359.363579
4,FDY38,OUT027,5730.875120


### Stacking

In [151]:
print(RMSEs)

{'Ridge': 26406.869688797884, 'DT': 31979.03245721867, 'RF': 10745.33626169421, 'KNN': 35189.87401966457, 'Ada': 31331.950847285876, 'LGBM': 15191.039788656703}


#### Voting Regressor

In [49]:
votreg = VotingRegressor(estimators = [('rf', rf), ('lgbr', lgbr), ('gbm', gbm)])

votreg.fit(X_train, y_train)

y_train_pred = votreg.predict(X_train)
print("Voting Regressor RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))

Voting Regressor RMSE: 1.8682133440244961


In [50]:
# Preparing the data for submission
y_test_pred = votreg.predict(X_test)

solution = pd.DataFrame({'Item_Identifier' : test.Item_Identifier, 
                        'Outlet_Identifier' : test.Outlet_Identifier,
                        'Item_Outlet_Sales' : y_test_pred ** 3})

# Export to CSV
solution.to_csv("Solution8_BigMart_VotReg_Saurabh.csv", index = False)

display(solution.head())

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1501.160525
1,FDW14,OUT017,1302.891282
2,NCN55,OUT010,559.381528
3,FDQ58,OUT017,2378.760525
4,FDY38,OUT027,5917.096665
